Import Dependancies

In [16]:
# Dependencies
from config import (gkey)
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

#import cleaned county data
county_data = pd.read_csv("../Matthew/Data_csv/state_county.csv")
county_data.loc[(county_data['County'] == 'Christian'),:]

,Unnamed: 0,State,County,"Male life expectancy, 1985 (years)","Female life expectancy, 1985 (years)","Male life expectancy, 1990 (years)","Female life expectancy, 1990 (years)","Male life expectancy, 1995 (years)","Female life expectancy, 1995 (years)","Male life expectancy, 2000 (years)","Female life expectancy, 2000 (years)","Male life expectancy, 2005 (years)","Female life expectancy, 2005 (years)","Male life expectancy, 2010 (years)","Female life expectancy, 2010 (years)","Difference in male life expectancy, 1985-2010 (years)","Difference in female life expectancy, 1985-2010 (years)"
617,619,Illinois,Christian,71.3,78.6,72.3,78.9,72.9,79.0,73.8,78.8,73.7,79.6,75.3,79.8,4.0,1.1
1032,1034,Kentucky,Christian,69.6,77.3,70.8,77.3,71.1,77.7,71.1,77.0,71.5,77.0,72.2,77.7,2.6,0.5
1528,1530,Missouri,Christian,72.1,79.1,72.9,80.0,74.5,80.2,74.2,79.7,75.0,79.9,77.4,80.8,5.3,1.7


In [2]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

# Generate Random Cities in the USA

In [3]:
# Range of latitudes and longitudes
lat_range = (25, 50)
lng_range = (-125,-65)

# List for holding lat_lngs and cities
lat_lngs = []
random_cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in random_cities:
        random_cities.append(city)

# Print the city count to confirm sufficient count
#random_cities

# Gather Geo and County Data

In [4]:
#test space for json

# Build the endpoint URL
test_url = f"https://maps.googleapis.com/maps/api/geocode/json?address=houston&key={gkey}"

test_data = requests.get(test_url).json()
test_data['results'][0]['address_components'][2]['long_name']

'Texas'

In [22]:
# Run a request to endpoint and convert result to json
#geo_data = requests.get(target_url).json()

cities = []
state = []
county = []
lat = []
lon = []

# Print the json
#grab county from the json
for city in random_cities[:1]:
    # create query url and get json
    query_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={city}&key={gkey}"
    geo_data = requests.get(query_url).json()
    #verify = geo_data['results'][0]['address_components'][3]['short_name']
    
    #if the city has data then populate lists
    if (geo_data['status'] == 'OK'):
    
        try:
            geo_data['results'][0]['address_components'][3]['short_name'] == 'US'
            state.append(geo_data['results'][0]['address_components'][2]['long_name'])
            county.append(geo_data['results'][0]['address_components'][1]['long_name'].replace('County',''))
            lat.append(geo_data["results"][0]["geometry"]["location"]["lat"])
            lon.append(geo_data["results"][0]["geometry"]["location"]["lng"])
            cities.append(city)
            
        except IndexError:
            print(city)
            pass
    else:
        next

In [23]:
len(cities)

1

In [24]:
len(lon)

1

In [25]:
len(county)

1

In [26]:
len(cities)

1

In [38]:
city_geo_data = pd.DataFrame({
    "City": cities,
    "State": state,
    "County": county,
    "Latitude": lat,
    "Longitude": lon
})

city_geo_data

,City,State,County,Latitude,Longitude
0,taylorville,Illinois,Christian,39.548935,-89.294533


In [39]:
county_data.loc[(county_data['County'] == 'Christian'),:]

,Unnamed: 0,State,County,"Male life expectancy, 1985 (years)","Female life expectancy, 1985 (years)","Male life expectancy, 1990 (years)","Female life expectancy, 1990 (years)","Male life expectancy, 1995 (years)","Female life expectancy, 1995 (years)","Male life expectancy, 2000 (years)","Female life expectancy, 2000 (years)","Male life expectancy, 2005 (years)","Female life expectancy, 2005 (years)","Male life expectancy, 2010 (years)","Female life expectancy, 2010 (years)","Difference in male life expectancy, 1985-2010 (years)","Difference in female life expectancy, 1985-2010 (years)"
617,619,Illinois,Christian,71.3,78.6,72.3,78.9,72.9,79.0,73.8,78.8,73.7,79.6,75.3,79.8,4.0,1.1
1032,1034,Kentucky,Christian,69.6,77.3,70.8,77.3,71.1,77.7,71.1,77.0,71.5,77.0,72.2,77.7,2.6,0.5
1528,1530,Missouri,Christian,72.1,79.1,72.9,80.0,74.5,80.2,74.2,79.7,75.0,79.9,77.4,80.8,5.3,1.7


# Merge Data

In [51]:
heatmap_data = pd.merge(city_geo_data, county_data, on=['State','County'], how = 'outer')
heatmap_data.loc[(county_data['County'] == 'Christian'),:]

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).